In [2]:
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from typing import TypedDict
from dotenv import load_dotenv

In [9]:
load_dotenv()
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)


In [10]:
class Blogstate(TypedDict):
    title: str
    outline:str
    content:str

In [11]:
def create_outline(state: Blogstate) -> Blogstate:
    title = state['title']

    prompt = f'generate a detailed outline for a blog on the topic - {title}'
    outline = llm.invoke(prompt).content

    state['outline'] = outline

    return state

In [12]:
def create_blog(state: Blogstate) -> Blogstate:
    title = state['title']
    outline = state['outline']

    prompt = f'write a detailed blog on title - {title} using the following outline - {outline}'
    content = llm.invoke(prompt).content
    state['content'] = content

In [13]:
graph = StateGraph(Blogstate)

graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)

graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)
workflow = graph.compile()

In [14]:
intial_state = {'title': 'the rise of ai in india'}
final_state = workflow.invoke(intial_state)
print(final_state)

{'title': 'the rise of ai in india', 'outline': '**Title:** "The Rise of AI in India: Opportunities, Challenges, and Future Prospects"\n\n**I. Introduction**\n\n* Brief overview of the growing importance of Artificial Intelligence (AI) globally\n* Significance of India\'s AI landscape\n* Thesis statement: India is poised to become a major player in the global AI market, driven by government initiatives, technological advancements, and entrepreneurial spirit.\n\n**II. Government Initiatives and Policies**\n\n* Overview of the Indian government\'s AI strategy and policies\n* Key initiatives:\n\t+ National AI Portal\n\t+ AI for All program\n\t+ AI for Social Good initiative\n\t+ Data Governance Framework\n* Analysis of the impact of these initiatives on the Indian AI ecosystem\n\n**III. Technological Advancements**\n\n* Overview of the key AI technologies being developed in India\n* Focus on:\n\t+ Machine Learning (ML)\n\t+ Natural Language Processing (NLP)\n\t+ Computer Vision\n\t+ Robot